### 데이터 및 유틸리티 함수 로드

In [21]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩
digits = load_digits()

print(digits.data)
print("### digits.data.shape:", digits.data.shape)
print(digits.target)
print("### digits.target.shape:", digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
### digits.data.shape: (1797, 64)
[0 1 2 ... 8 9 8]
### digits.target.shape: (1797,)


In [22]:
# TODO: y 데이터 이진 라벨로 변환
y = (digits.target == 1).astype(int)

### Train / Test 데이터 나누기
- test_size=0.2
- random_state=0
- 클래스가 1 이면

In [38]:
# TODO
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, test_size=0.2, stratify=y, random_state=0)

In [40]:
print("X_train: {}".format(X_train.shape))
print("X_test: {}".format(X_test.shape))
print("y_train: {}".format(y_train.shape))
print("y_test: {}".format(y_test.shape))

X_train: (1437, 64)
X_test: (360, 64)
y_train: (1437,)
y_test: (360,)


### 모델 학습 및 평가
 - 대상 모델: 랜덤 포레스트(sklearn,ensemble,RandomForestClassifier, random_state=0), 로지스틱 회귀(LogisticRegression)
 - 평가 기준: 정확도

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve

# TODO: 랜덤 포레스트, 로지스틱 회귀 모델 학습
rf_clf = RandomForestClassifier(random_state=0)
dt_clf = DecisionTreeClassifier(random_state=0)

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train , y_train)
rf_acc_train = accuracy_score(y_train, rf_clf.predict(X_train))
rf_acc_test = accuracy_score(y_test, rf_clf.predict(X_test))
print('RandomForestClassifier Train 정확도:{0:.4f}'.format(rf_acc_train))
print('RandomForestClassifier Test 정확도:{0:.4f}'.format(rf_acc_test))

# LogisticRegression 학습/예측/평가
dt_clf.fit(X_train , y_train)
dt_acc_train = accuracy_score(y_train, dt_clf.predict(X_train))
dt_acc_test = accuracy_score(y_test, dt_clf.predict(X_test))
print('DecisionTree Train 정확도:{0:.4f}'.format(dt_acc_train))
print('DecisionTree Test 정확도:{0:.4f}'.format(dt_acc_test))


RandomForestClassifier Train 정확도:1.0000
RandomForestClassifier Test 정확도:0.9972
DecisionTree Train 정확도:1.0000
DecisionTree Test 정확도:0.9694


### DecisionTree 하이퍼파라미터 최적화
 - Target: max_depth (2, 3, 4, 5), min_samples_split (2, 4, 6, 8, 10)
 - 5-fold cross validation
 - 최적화 기준: 정확도, 정밀도, 재현율, F1

In [42]:
from sklearn.model_selection import GridSearchCV


In [43]:
# TODO: DecisionTree, Accuracy 기준 최적화
dt_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'max_depth':[2, 3, 4, 5]}

grid_dt_accuracy = GridSearchCV(dt_clf , param_grid=dt_parameters , scoring='accuracy' , cv=5)
grid_dt_accuracy.fit(X_train , y_train)

print('Accuracy 기준 DecisionTree 최적 하이퍼 파라미터 :',grid_dt_accuracy.best_params_)
print('Accuracy 기준 DecisionTree 최고 성능: {0:.4f}'.format(grid_dt_accuracy.best_score_))
best_dt_accuracy = grid_dt_accuracy.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_dt_accuracy_perfacc = accuracy_score(y_test, best_dt_accuracy.predict(X_test))
best_dt_accuracy_perfprec = precision_score(y_test, best_dt_accuracy.predict(X_test))
best_dt_accuracy_perfrec = recall_score(y_test, best_dt_accuracy.predict(X_test))
best_dt_accuracy_perff1 = f1_score(y_test, best_dt_accuracy.predict(X_test))

print('Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_dt_accuracy_perfacc))
print('Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_dt_accuracy_perfprec))
print('Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_dt_accuracy_perfrec))
print('Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_dt_accuracy_perff1))


Accuracy 기준 DecisionTree 최적 하이퍼 파라미터 : {'max_depth': 4, 'min_samples_split': 2}
Accuracy 기준 DecisionTree 최고 성능: 0.9701
Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : 0.9611
Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : 0.7895
Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : 0.8333
Accuracy 기준 DecisionTree 최적 모델의 테스트 세트 F1 : 0.8108


In [44]:
# TODO: DecisionTree, Precision 기준 최적화
dt_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'max_depth':[2, 3, 4, 5]}

grid_dt_precision = GridSearchCV(dt_clf , param_grid=dt_parameters , scoring='precision' , cv=5)
grid_dt_precision.fit(X_train , y_train)

print('Precision 기준 DecisionTree 최적 하이퍼 파라미터 :',grid_dt_precision.best_params_)
print('Precision 기준 DecisionTree 최고 성능: {0:.4f}'.format(grid_dt_precision.best_score_))
best_dt_precision = grid_dt_precision.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_dt_precision_perfacc = accuracy_score(y_test, best_dt_precision.predict(X_test))
best_dt_precision_perfprec = precision_score(y_test, best_dt_precision.predict(X_test))
best_dt_precision_perfrec = recall_score(y_test, best_dt_precision.predict(X_test))
best_dt_precision_perff1 = f1_score(y_test, best_dt_precision.predict(X_test))

print('Precision 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_dt_precision_perfacc))
print('Precision 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_dt_precision_perfprec))
print('Precision 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_dt_precision_perfrec))
print('Precision 기준 DecisionTree 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_dt_precision_perff1))


Precision 기준 DecisionTree 최적 하이퍼 파라미터 : {'max_depth': 5, 'min_samples_split': 10}
Precision 기준 DecisionTree 최고 성능: 0.8705
Precision 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : 0.9639
Precision 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : 0.7949
Precision 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : 0.8611
Precision 기준 DecisionTree 최적 모델의 테스트 세트 F1 : 0.8267


In [45]:
# TODO: DecisionTree, Recall 기준 최적화
dt_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'max_depth':[2, 3, 4, 5]}

grid_dt_recall = GridSearchCV(dt_clf , param_grid=dt_parameters , scoring='recall' , cv=5)
grid_dt_recall.fit(X_train , y_train)

print('Recall 기준 DecisionTree 최적 하이퍼 파라미터 :',grid_dt_recall.best_params_)
print('Recall 기준 DecisionTree 최고 성능: {0:.4f}'.format(grid_dt_recall.best_score_))
best_dt_recall = grid_dt_recall.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_dt_recall_perfacc = accuracy_score(y_test, best_dt_recall.predict(X_test))
best_dt_recall_perfprec = precision_score(y_test, best_dt_recall.predict(X_test))
best_dt_recall_perfrec = recall_score(y_test, best_dt_recall.predict(X_test))
best_dt_recall_perff1 = f1_score(y_test, best_dt_recall.predict(X_test))

print('Recall 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_dt_recall_perfacc))
print('Recall 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_dt_recall_perfprec))
print('Recall 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_dt_recall_perfrec))
print('Recall 기준 DecisionTree 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_dt_recall_perff1))


Recall 기준 DecisionTree 최적 하이퍼 파라미터 : {'max_depth': 5, 'min_samples_split': 2}
Recall 기준 DecisionTree 최고 성능: 0.8497
Recall 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : 0.9750
Recall 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : 0.8857
Recall 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : 0.8611
Recall 기준 DecisionTree 최적 모델의 테스트 세트 F1 : 0.8732


In [46]:
# TODO: DecisionTree, F1 기준 최적화
dt_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'max_depth':[2, 3, 4, 5]}

grid_dt_f1 = GridSearchCV(dt_clf , param_grid=dt_parameters , scoring='f1' , cv=5)
grid_dt_f1.fit(X_train , y_train)

print('F1 기준 DecisionTree 최적 하이퍼 파라미터 :',grid_dt_f1.best_params_)
print('F1 기준 DecisionTree 최고 성능: {0:.4f}'.format(grid_dt_f1.best_score_))
best_dt_f1 = grid_dt_f1.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_dt_f1_perfacc = accuracy_score(y_test, best_dt_f1.predict(X_test))
best_dt_f1_perfprec = precision_score(y_test, best_dt_f1.predict(X_test))
best_dt_f1_perfrec = recall_score(y_test, best_dt_f1.predict(X_test))
best_dt_f1_perff1 = f1_score(y_test, best_dt_f1.predict(X_test))

print('F1 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_dt_f1_perfacc))
print('F1 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_dt_f1_perfprec))
print('F1 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_dt_f1_perfrec))
print('F1 기준 DecisionTree 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_dt_f1_perff1))


F1 기준 DecisionTree 최적 하이퍼 파라미터 : {'max_depth': 4, 'min_samples_split': 2}
F1 기준 DecisionTree 최고 성능: 0.8532
F1 기준 DecisionTree 최적 모델의 테스트 세트 정확도 : 0.9611
F1 기준 DecisionTree 최적 모델의 테스트 세트 정밀도 : 0.7895
F1 기준 DecisionTree 최적 모델의 테스트 세트 재현율 : 0.8333
F1 기준 DecisionTree 최적 모델의 테스트 세트 F1 : 0.8108


### RandomForest 하이퍼파라미터 최적화
 - Target: min_samples_split (2, 4, 6, 8, 10), n_estimators(100, 300, 500)
 - 5-fold cross validation
 - 최적화 기준: 정확도, 정밀도, 재현율, F1

In [47]:
from sklearn.model_selection import GridSearchCV


In [48]:
rf_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'n_estimators':[100, 300, 500]}

grid_rf_accuracy = GridSearchCV(rf_clf , param_grid=rf_parameters , scoring='accuracy' , cv=5)
grid_rf_accuracy.fit(X_train , y_train)

print('Accuracy 기준 RandomForest 최적 하이퍼 파라미터 :',grid_rf_accuracy.best_params_)
print('Accuracy 기준 RandomForest 최고 성능: {0:.4f}'.format(grid_rf_accuracy.best_score_))
best_rf_accuracy = grid_rf_accuracy.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_rf_accuracy_perfacc = accuracy_score(y_test, best_rf_accuracy.predict(X_test))
best_rf_accuracy_perfprec = precision_score(y_test, best_rf_accuracy.predict(X_test))
best_rf_accuracy_perfrec = recall_score(y_test, best_rf_accuracy.predict(X_test))
best_rf_accuracy_perff1 = f1_score(y_test, best_rf_accuracy.predict(X_test))

print('Accuracy 기준 RandomForest 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_rf_accuracy_perfacc))
print('Accuracy 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_rf_accuracy_perfprec))
print('Accuracy 기준 RandomForest 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_rf_accuracy_perfrec))
print('Accuracy 기준 RandomForest 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_rf_accuracy_perff1))


Accuracy 기준 RandomForest 최적 하이퍼 파라미터 : {'min_samples_split': 4, 'n_estimators': 100}
Accuracy 기준 RandomForest 최고 성능: 0.9958
Accuracy 기준 RandomForest 최적 모델의 테스트 세트 정확도 : 0.9972
Accuracy 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : 1.0000
Accuracy 기준 RandomForest 최적 모델의 테스트 세트 재현율 : 0.9722
Accuracy 기준 RandomForest 최적 모델의 테스트 세트 F1 : 0.9859


In [49]:
# TODO: RandomFrest, Precision 기준 최적화

rf_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'n_estimators':[100, 300, 500]}

grid_rf_prec = GridSearchCV(rf_clf , param_grid=rf_parameters , scoring='precision' , cv=5)
grid_rf_prec.fit(X_train , y_train)

print('Precision 기준 RandomForest 최적 하이퍼 파라미터 :',grid_rf_prec.best_params_)
print('Precision 기준 RandomForest 최고 성능: {0:.4f}'.format(grid_rf_prec.best_score_))
best_rf_prec = grid_rf_prec.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_rf_prec_perfacc = accuracy_score(y_test, best_rf_prec.predict(X_test))
best_rf_prec_perfprec = precision_score(y_test, best_rf_prec.predict(X_test))
best_rf_prec_perfrec = recall_score(y_test, best_rf_prec.predict(X_test))
best_rf_prec_perff1 = recall_score(y_test, best_rf_prec.predict(X_test))

print('Precision 기준 RandomForest 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_rf_prec_perfacc))
print('Precision 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_rf_prec_perfprec))
print('Precision 기준 RandomForest 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_rf_prec_perfrec))
print('Precision 기준 RandomForest 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_rf_prec_perff1))


Precision 기준 RandomForest 최적 하이퍼 파라미터 : {'min_samples_split': 4, 'n_estimators': 100}
Precision 기준 RandomForest 최고 성능: 1.0000
Precision 기준 RandomForest 최적 모델의 테스트 세트 정확도 : 0.9972
Precision 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : 1.0000
Precision 기준 RandomForest 최적 모델의 테스트 세트 재현율 : 0.9722
Precision 기준 RandomForest 최적 모델의 테스트 세트 F1 : 0.9722


In [50]:
# TODO: RandomForest, Recall 기준 최적화

rf_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'n_estimators':[100, 300, 500]}

grid_rf_rec = GridSearchCV(rf_clf , param_grid=rf_parameters , scoring='recall' , cv=5)
grid_rf_rec.fit(X_train , y_train)

print('Recall 기준 RandomForest 최적 하이퍼 파라미터 :',grid_rf_rec.best_params_)
print('Recall 기준 RandomForest 최고 성능: {0:.4f}'.format(grid_rf_rec.best_score_))
best_rf_rec = grid_rf_rec.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_rf_rec_perfacc = accuracy_score(y_test, best_rf_rec.predict(X_test))
best_rf_rec_perfprec = precision_score(y_test, best_rf_rec.predict(X_test))
best_rf_rec_perfrec = recall_score(y_test, best_rf_rec.predict(X_test))
best_rf_rec_perff1 = f1_score(y_test, best_rf_rec.predict(X_test))

print('Recall 기준 RandomForest 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_rf_rec_perfacc))
print('Recall 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_rf_rec_perfprec))
print('Recall 기준 RandomForest 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_rf_rec_perfrec))
print('Recall 기준 RandomForest 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_rf_rec_perff1))


Recall 기준 RandomForest 최적 하이퍼 파라미터 : {'min_samples_split': 2, 'n_estimators': 300}
Recall 기준 RandomForest 최고 성능: 0.9591
Recall 기준 RandomForest 최적 모델의 테스트 세트 정확도 : 0.9944
Recall 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : 1.0000
Recall 기준 RandomForest 최적 모델의 테스트 세트 재현율 : 0.9444
Recall 기준 RandomForest 최적 모델의 테스트 세트 F1 : 0.9714


In [51]:
# TODO: RandomForest, F1 기준 최적화

rf_parameters = {'min_samples_split':[2, 4, 6, 8, 10], 'n_estimators':[100, 300, 500]}

grid_rf_f1 = GridSearchCV(rf_clf , param_grid=rf_parameters , scoring='f1' , cv=5)
grid_rf_f1.fit(X_train , y_train)

print('Recall 기준 RandomForest 최적 하이퍼 파라미터 :',grid_rf_rec.best_params_)
print('Recall 기준 RandomForest 최고 성능: {0:.4f}'.format(grid_rf_rec.best_score_))
best_rf_f1 = grid_rf_f1.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
best_rf_f1_perfacc = accuracy_score(y_test, best_rf_f1.predict(X_test))
best_rf_f1_perfprec = precision_score(y_test, best_rf_f1.predict(X_test))
best_rf_f1_perfrec = recall_score(y_test, best_rf_f1.predict(X_test))
best_rf_f1_perff1 = f1_score(y_test, best_rf_f1.predict(X_test))

print('F1 기준 RandomForest 최적 모델의 테스트 세트 정확도 : {0:.4f}'.format(best_rf_f1_perfacc))
print('F1 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : {0:.4f}'.format(best_rf_f1_perfprec))
print('F1 기준 RandomForest 최적 모델의 테스트 세트 재현율 : {0:.4f}'.format(best_rf_f1_perfrec))
print('F1 기준 RandomForest 최적 모델의 테스트 세트 F1 : {0:.4f}'.format(best_rf_f1_perff1))


Recall 기준 RandomForest 최적 하이퍼 파라미터 : {'min_samples_split': 2, 'n_estimators': 300}
Recall 기준 RandomForest 최고 성능: 0.9591
F1 기준 RandomForest 최적 모델의 테스트 세트 정확도 : 0.9972
F1 기준 RandomForest 최적 모델의 테스트 세트 정밀도 : 1.0000
F1 기준 RandomForest 최적 모델의 테스트 세트 재현율 : 0.9722
F1 기준 RandomForest 최적 모델의 테스트 세트 F1 : 0.9859
